# DataSet e bibliotecas a serem usados no projeto

In [28]:
import pandas as pd
import jax.numpy as jnp
import numpy as np
import jax as jax
from typing import Callable
from sklearn.model_selection import train_test_split
import time
import timeit
import matplotlib.pyplot as plt

# user id | item id | rating | timestamp.
_df = pd.read_csv('ml-100k/u.data', delimiter='\t', header=None, names=['userId', 'movieId', 'rating', 'timeStamp'])
print(_df.head())

   userId  movieId  rating  timeStamp
0     196      242       3  881250949
1     186      302       3  891717742
2      22      377       1  878887116
3     244       51       2  880606923
4     166      346       1  886397596


### Tratamento dos dados: transposição em matriz, preenchimento de dados faltantes, normalização

In [34]:
# Convertendo a lista de dados em uma tabela com usuários nas linhas, filmes na colunas, contendo os ratings correspondentes.
df = _df.pivot(index='userId', columns='movieId', values='rating')
print(df.head(), '\n')
print("Novo formato do DataFrame: ", df.shape, '\n')

# Preenchendo valores faltantes com o rating médio do filme correspondente
dfMovieMean = df.apply(lambda x: x.fillna(x.mean()), axis=0)

# Preenchendo valores faltantes com a média dos ratings dados pelo usuário
dfUserMean = df.apply(lambda x: x.fillna(x.mean()), axis=1)

# Preenchendo valroes faltantes com zeros
dfZeros = df.fillna(0)

print("DataFrame with column mean filled:\n", dfMovieMean.head(), '\n')
print("DataFrame with row mean filled:\n", dfUserMean.head(), '\n')
print("DataFrame with zeros filled:\n", dfZeros.head(), '\n')

movieId  1     2     3     4     5     6     7     8     9     10    ...  \
userId                                                               ...   
1         5.0   3.0   4.0   3.0   3.0   5.0   4.0   1.0   5.0   3.0  ...   
2         4.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   2.0  ...   
3         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
4         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
5         4.0   3.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   

movieId  1673  1674  1675  1676  1677  1678  1679  1680  1681  1682  
userId                                                               
1         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
2         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
3         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
4         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
5         NaN   NaN   NaN   NaN   NaN   NaN   N

# Separação dos dados de treino e teste, normalização e conversão para arrays de JAX

In [ ]:
def normalizeData(data:jnp.ndarray) -> jnp.ndarray:
    """
    Normaliza dados de um array subtraindo médias normalizando em relação ao desvio padrão de cada feature.

    Args:
        data (jnp.ndarray): Dados não normalizdos

    Returns:
        jnp.ndarray: Dados normalizados
    """
    #Normalizando os dados: subtração da média de cada feature e mapeando para o intervalo [0,1]
    means = jnp.mean(data, axis=0)
    normalizedData = data - means

    std = jnp.std(normalizedData, axis=0)
    normalizedData = normalizedData / std
    
    return normalizedData

# Separando dados de treino e de teste
features = dfZeros.columns.tolist()[:-1]
x = dfZeros[[feature for feature in features]]  
y = dfZeros['target']  

# Normalizando os dados. Como a função de normalização funciona com JAX, 
# mas a separação dos conjuntos de teste e treino é feita pelo scikit-learn, precisei fazer duas conversões entre tipos.
x = normalizeData(jnp.array(x.values))
x = pd.DataFrame(x)

xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)

# Convertendo para arrays compatíveis com JAX:
xTrain = jnp.array(xTrain)
xTest = jnp.array(xTest)
yTrain = jnp.array(yTrain)
yTest = jnp.array(yTest)

# Definindo as funções para implementação do PCA e KNN

In [ ]:
def pca(data:jnp.ndarray, nbComponents:int, svd=True) -> jnp.ndarray:
    """
    A partir de um conjunto de dados, retorna a matriz de projeção para as k componentes principais.

    Args:
        data (jnp.ndaaray): Dados para reduzir dimensionalidade.
        nbComponents (int): Número de componentes desejadas na projeção.

    Returns:
        jnp.ndarray: Matriz de projeção.
    """

    covarianceMatrix = (1/len(data)) * data.T @ data
    
    if svd:
        # Realizando a decomposição em valores singulares. U terá vetores coluna ortogonais.  
        # A pricípio, o ordenamento não é necessário pois svd() já retorna os valores singulares ordenados.  
        U, S, Ut = jax.scipy.linalg.svd(covarianceMatrix) 
        projectionMatrix = U[:, :nbComponents]
    else:
        # Decomposição a partir de autovalores e autovetores.
        # Aqui, o ordenamento dos autovetores baseado nos autovalores é necessário.
        eigenvalues, eigenvectors = jnp.linalg.eig(covarianceMatrix)  
        indices = jnp.argsort(eigenvalues, descending=True)           
        projectionMatrix = eigenvectors[:, indices]  
        projectionMatrix = projectionMatrix[:, :nbComponents]
   

    return projectionMatrix

In [ ]:
def knn(xTrain:jnp.ndarray, yTrain:jnp.ndarray, xTest:jnp.ndarray, k:int, metric:Callable[[jnp.ndarray, jnp.ndarray], float]) -> jnp.ndarray:
    """ Implementa a classificação de um conjunto de dados a partir do algoritmo de K-Nearest Neighbors (KNN).

    Args:
        xTrain (jnp.ndarray): Dados de treino
        yTrain (jnp.ndarray): Labels para os dados de treino
        xTest (jnp.ndarray): Dados de teste
        k (int): Número de vizinhos mais próximos usados para a classificação.
        metric (Callable): Função que calcula distância entre dois pontos.

    Returns:
        jnp.ndarray: Array contendo as predições realizadas para o conjunto de dados de teste xTest.
    """
    
    # Implementação JAX-friendly do cálculo da matriz de distâncias para um conjunto de pontos de teste.
    # O cálculo da matriz é feito vetorizando duas vezes a função de métrica, para calcular dois a dois
    # as distâncias entre pontos de treino e de teste.
    distances = jax.vmap(lambda train_point: jax.vmap(metric, in_axes=(None, 0))(train_point, xTest))(xTrain)

    # Ordenando as distâncias entre pontos, e pegando os targets/labels dos k pontos mais próximos para cada ponto de treino
    sorted_indices = jnp.argsort(distances, axis=0)
    nearestNeighborsIndices = sorted_indices[:k, :]
    nearestNeighbors = yTrain[nearestNeighborsIndices].astype(int)

    # Contagem dos números de cada target presente na lista de vizinhos próximos e definição do target estimado para o ponto
    # de teste baseado numa voto de maioria.
    totalLabels = 3
    targetCounts = jax.vmap(lambda neighbors: jnp.bincount(neighbors, minlength=totalLabels, length=3))(nearestNeighbors.T) 
    most_common_classes = jnp.argmax(targetCounts, axis=1)
    
    return most_common_classes